In [121]:
import os 
import pandas as pd
import subprocess
import glob
import pybedtools as pbt 

pbt.set_bedtools_path('/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/')

gsizes = 'results/refs/hg19/hg19.chrom.sizes'
res = 10000

os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')

# make the directory to save our data
outdir = 'results/main/gwas_pieqtls/2021_chiou_et_al/2021_chandra_et_al/'
os.makedirs(outdir, exist_ok=True)
bedpe_cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB']

In [122]:
chain="/mnt/BioHome/jreyna/jreyna/projects/dchallenge/results/main/2021_Nikhil_eQTL/Data/refs/ucsc/hg38ToHg19.over.chain.gz"

In [123]:
def liftover(chain, bed, lifted, unmapped):
    cmd = '/mnt/BioHome/jreyna/software/UCSC_Browser_Tools/liftOver -bedPlus=3 -tab {} {} {} {}'
    cmd = cmd.format(bed, chain, lifted, unmapped)
    sp = subprocess.Popen(cmd, stderr=subprocess.STDOUT, shell=True)
    print(sp.communicate())

## Load Coloc Datasets

In [124]:
colocs = 'results/main/coloc/Results/Colocalization_SMKN/T1D_34012112_Gaulton/'
colocs += '*/*/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
colocs = glob.glob(colocs)

In [125]:
coloc_data = []
for coloc in colocs:

    cline = coloc.split('/')[7]
    
    lifted = os.path.join(outdir, '{}.lifted.bed'.format(cline))
    unmapped = os.path.join(outdir, '{}.unmapped.bed'.format(cline))
    
    if not os.path.exists(lifted):
        
        df = pd.read_table(coloc, header=0)   
        df['start'] = df['pos'] - 1
        df = df.iloc[:, [0,-1,1] + list(range(2, 24))]

        interm_fn = os.path.join(outdir, '{}.interm.bed'.format(cline))
        df.to_csv(interm_fn, sep='\t', index=False, header=False)    
        liftover(chain, interm_fn, lifted, unmapped)
    
    lifted_df = pd.read_table(lifted, header=None)

    # add cell type
    lifted_df['cline'] = cline

    coloc_data.append(lifted_df) 


Reading liftover chains
Mapping coordinates
(None, None)
Reading liftover chains
Mapping coordinates
(None, None)
Reading liftover chains
Mapping coordinates
(None, None)


In [126]:
coloc_df = pd.concat(coloc_data)

In [127]:
coloc_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,cline
0,chr16,75281234,75281235,5.465817e-64,3.551930e-64,0.070681,0.045048,0.884271,rs11317724,rs11317724:75247337:GT:G,...,G,700,0.139776,5008,0.021805,0.170677,0.022578,4.040000e-14,520580,T-cell
1,chr16,75281234,75281235,5.465817e-64,3.551930e-64,0.070681,0.045048,0.884271,rs11317724,rs11317724:75247337:GT:G,...,G,700,0.139776,5008,5.915775,0.170677,0.022578,4.040000e-14,520580,T-cell
2,chr16,75281234,75281235,5.465817e-64,3.551930e-64,0.070681,0.045048,0.884271,rs11317724,rs11317724:75247337:GT:G,...,G,700,0.139776,5008,31.589022,0.170677,0.022578,4.040000e-14,520580,T-cell
3,chr16,75281234,75281235,5.465817e-64,3.551930e-64,0.070681,0.045048,0.884271,rs11317724,rs11317724:75247337:GT:G,...,G,700,0.139776,5008,3.780248,0.170677,0.022578,4.040000e-14,520580,T-cell
4,chr16,75281234,75281235,5.465817e-64,3.551930e-64,0.070681,0.045048,0.884271,rs11317724,rs11317724:75247337:GT:G,...,G,700,0.139776,5008,0.852248,0.170677,0.022578,4.040000e-14,520580,T-cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,chr17,44227509,44227510,0.000000e+00,0.000000e+00,0.045706,0.094956,0.859337,rs35841265,rs35841265:46150144:C:CA,...,CA,1290,0.257588,5008,0.283795,-0.094581,0.017349,4.990000e-08,520580,blood
32,chr17,44227509,44227510,0.000000e+00,0.000000e+00,0.045706,0.094956,0.859337,rs35841265,rs35841265:46150144:C:CA,...,CA,1290,0.257588,5008,0.154602,-0.094581,0.017349,4.990000e-08,520580,blood
33,chr17,44227509,44227510,0.000000e+00,0.000000e+00,0.045706,0.094956,0.859337,rs35841265,rs35841265:46150144:C:CA,...,CA,1290,0.257588,5008,1.707966,-0.094581,0.017349,4.990000e-08,520580,blood
34,chr17,44227509,44227510,0.000000e+00,0.000000e+00,0.045706,0.094956,0.859337,rs35841265,rs35841265:46150144:C:CA,...,CA,1290,0.257588,5008,0.001409,-0.094581,0.017349,4.990000e-08,520580,blood


In [128]:
coloc_bed = coloc_df.iloc[:, [0,1,2,-1]]
coloc_bed.columns = ['chr', 'start', 'end', 'cline']
coloc_bed['chr'] = coloc_bed['chr'].str.replace('chr', '')

/tmp/ipykernel_36935/1847490033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coloc_bed['chr'] = coloc_bed['chr'].str.replace('chr', '')


## Intersect Colocs and pieQTLs

In [159]:
pieqtls = glob.glob('results/main/pieqtls/2021_chandra_et_al/*/proximal.pieqtls.tsv')

In [160]:
pieqtl_data = []
for pieqtl in pieqtls:
    print(pieqtl)
    
    cline = pieqtl.split('/')[-2]
    df = pd.read_table(pieqtl)    
    df['cline'] = cline

    pieqtl_data.append(df) 

results/main/pieqtls/2021_chandra_et_al/B-cell_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/NK-cell_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/monocyte_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/CD4_T-cell_naive/proximal.pieqtls.tsv
results/main/pieqtls/2021_chandra_et_al/CD8_T-cell_naive/proximal.pieqtls.tsv


In [161]:
pieqtl_df = pd.concat(pieqtl_data)

In [162]:
pieqtl_bed = pieqtl_df.iloc[:, [1,2,2,-1]]
pieqtl_bed.columns = ['chrom', 'start', 'end', 'cline']
pieqtl_bed.loc[:, 'chrom'] = pieqtl_bed['chrom'].str.replace('chr', '')
pieqtl_bed.loc[:, 'start'] = pieqtl_bed['start'] - 1

/tmp/ipykernel_36935/4052474229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pieqtl_bed.loc[:, 'chrom'] = pieqtl_bed['chrom'].str.replace('chr', '')
/tmp/ipykernel_36935/4052474229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pieqtl_bed.loc[:, 'start'] = pieqtl_bed['start'] - 1


In [163]:
pieqtl_to_coloc_comps = [('B-cell_naive', 'B-cell_naive'),
                      ('NK-cell_naive', 'NK-cell_naive'),
                      ('monocyte_naive', 'monocyte_CD16_naive'), 
                      ('monocyte_naive', 'monocyte_naive'),
                      ('monocyte_naive', 'monocyte_LPS'),
                      ('monocyte_naive', 'monocyte_IAV'),
                      ('monocyte_naive', 'monocyte_Pam3CSK4'),
                      ('CD4_T-cell_naive', 'CD4_T-cell_anti-CD3-CD28'), 
                      ('CD4_T-cell_naive', 'CD4_T-cell_naive'),  
                      ('CD8_T-cell_naive', 'CD8_T-cell_naive'), 
                      ('CD8_T-cell_naive', 'CD8_T-cell_anti-CD3-CD28')]

In [164]:
pc_df = pd.DataFrame(pieqtl_to_coloc_comps)

In [165]:
pc_df.columns = ['pieQTL_source', 'coloc_source']

In [166]:
pc_df

,pieQTL_source,coloc_source
0,B-cell_naive,B-cell_naive
1,NK-cell_naive,NK-cell_naive
2,monocyte_naive,monocyte_CD16_naive
3,monocyte_naive,monocyte_naive
4,monocyte_naive,monocyte_LPS
5,monocyte_naive,monocyte_IAV
6,monocyte_naive,monocyte_Pam3CSK4
7,CD4_T-cell_naive,CD4_T-cell_anti-CD3-CD28
8,CD4_T-cell_naive,CD4_T-cell_naive
9,CD8_T-cell_naive,CD8_T-cell_naive


In [167]:
pieqtl_bed_grps = pieqtl_bed.groupby('cline')
coloc_bed_grps = coloc_bed.groupby('cline')

In [177]:
intersection_data = []
for pieqtl_cell, coloc_cell in pieqtl_to_coloc_comps: 
    
    # skip TBDs
    if 'TBD' in [pieqtl_cell, coloc_cell]:
        continue
    
    pieqtl_cline = pieqtl_bed_grps.get_group(pieqtl_cell)
    coloc_cline = coloc_bed_grps.get_group(coloc_cell)
    
    pieqtl_pbt = pbt.BedTool.from_dataframe(pieqtl_cline)
    coloc_pbt = pbt.BedTool.from_dataframe(coloc_cline)
    
    intersection = pieqtl_pbt.intersect(coloc_pbt.slop(b=200000, g=gsizes), wa=True, wb=True).to_dataframe()
    
    if len(intersection) > 0: 
        intersection_data.append(intersection)
        print(pieqtl_cell, coloc_cell)


monocyte_naive monocyte_naive
monocyte_naive monocyte_IAV


In [178]:
intersection_df = pd.concat(intersection_data)

In [179]:
intersection_df

,chrom,start,end,name,score,strand,thickStart,thickEnd
0,18,12947615,12947616,monocyte_naive,18,12580462,12980463,monocyte_naive
1,18,12947615,12947616,monocyte_naive,18,12580462,12980463,monocyte_naive
2,18,12947615,12947616,monocyte_naive,18,12580462,12980463,monocyte_naive
3,18,12947615,12947616,monocyte_naive,18,12580462,12980463,monocyte_naive
4,18,12947615,12947616,monocyte_naive,18,12580462,12980463,monocyte_naive
...,...,...,...,...,...,...,...,...
70,18,12947461,12947462,monocyte_naive,18,12580462,12980463,monocyte_IAV
71,18,12947461,12947462,monocyte_naive,18,12580462,12980463,monocyte_IAV
72,18,12947461,12947462,monocyte_naive,18,12580462,12980463,monocyte_IAV
73,18,12947461,12947462,monocyte_naive,18,12580462,12980463,monocyte_IAV
